<a href="https://colab.research.google.com/github/dolDolSee/TF2.0/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
#딥러닝에서는 실수형을 사용하기 때문에 int형에서 변경해줌
x_train, x_test = x_train.astype("float32"), x_test.astype("float32")
#flatting: 2차원 메트릭스를 1차원 벡터의 784(28*28)으로 변환해줌
x_train, x_test = x_train.reshape([-1,784]), x_test.reshape([-1,784])
x_train, x_test = x_train/255., x_test /255.

y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)



In [5]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(50)
train_data_iter = iter(train_data)

In [ ]:
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()
    #padding=same의 의미는 원본사이즈와 동일한 사이즈의 feature map이 나오도록 강제하겠다. 라는 의미임.
    self.conv_layer_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1, padding="same", activation="relu")
    self.pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)

    self.conv_layer_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)

    self.flatten_layer = tf.keras.layers.Flatten()
    self.fc_layer_1 = tf.keras.layers.Dense(1024, activation='relu')

    self.output_layer = tf.keras.layers.Dense(10, activation=None)
  
  def call(self, x):
    x_image = tf.reshape(x,[-1, 28,28,1])
    # 28*28*1 -> 28*28*32
    h_conv1 = self.conv_layer_1(x_image)
    # 28*28*32 -> 14*14*32
    h_pool1 = self.pool_layer_1(h_conv1)
    # 14*14*32 -> 14*14*64
    h_conv2 = self.conv_layer_2(h_pool1)
    # 14*14*64 -> 7*7*64
    h_pool2 = self.pool_layer_2(h_conv2)
    # 7*7*64(3136) -> 1024
    h_pool2_flat = self.flatten_layer(h_pool2)
    print(h_pool2_flat)
    h_fc1 = self.fc_layer_1(h_pool2_flat)
    # 1024 -> 10
    logits = self.output_layer(h_fc1)
    y_pred = tf.nn.softmax(logits)

    return y_pred, logits

@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

optimizer = tf.optimizers.Adam(1e-4)

@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred, logits = model(x)
    loss = cross_entropy_loss(logits,y_pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

CNN_model = CNN()

for i in range(10000):
  batch_x, batch_y = next(train_data_iter)
  if i % 100 ==0:
    train_accuracy = compute_accuracy(CNN_model(batch_x)[0], batch_y)
    print(train_accuracy)

  train_step(CNN_model, batch_x, batch_y)

print(compute_accuracy(x_test)[0],y_test)

In [1]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train.astype('float32'), x_test.astype("float32")
x_train, x_test = x_train/255., x_test/255.

#squeeze는 더미 dimension을 없애는 api
y_train_one_hot = tf.squeeze(tf.one_hot(y_train,10), axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test,10), axis=1)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train_one_hot))
train_data = train_data.repeat().shuffle(50000).batch(128)
train_data_iter = iter(train_data)

test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test_one_hot))
test_data = test_data.batch(1000)
test_data_iter = iter(test_data)

170508288/170498071 [==============================] - 6s 0us/step


In [2]:
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv_layer_1 = tf.keras.layers.Conv2D(filters=64, kernel_size = 5, strides=1, padding="same", activation="relu")
    self.Pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)
    #64개의 특징을 64개의 특징으로 맵핑
    self.conv_layer_2 =  tf.keras.layers.Conv2D(filters=64, kernel_size = 5, strides=1, padding="same", activation="relu")
    self.Pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)

    self.conv_layer_3 =  tf.keras.layers.Conv2D(filters=128, kernel_size = 3, strides=1, padding="same", activation="relu")
    self.conv_layer_4 =  tf.keras.layers.Conv2D(filters=128, kernel_size = 3, strides=1, padding="same", activation="relu")
    self.conv_layer_5 =  tf.keras.layers.Conv2D(filters=128, kernel_size = 3, strides=1, padding="same", activation="relu")

    #8*8*128의 feature map을 flatten 후 384개의 특징들로 맵핑한다.
    self.flatten_layer = tf.keras.layers.Flatten()
    self.fc_layer_1 = tf.keras.layers.Dense(384, activation="relu")

    self.dropout = tf.keras.layers.Dropout(0.2)

    #label 수 만틈 나가는 노드 지정
    self.output_layer = tf.keras.layers.Dense(10, activation=None)
  
  def call(self, x, is_training):
    h_conv1 = self.conv_layer_1(x)
    h_pool1 = self.Pool_layer_1(h_conv1)
    h_conv2 = self.conv_layer_2(h_pool1)
    h_pool2 = self.Pool_layer_2(h_conv2)
    h_conv3 = self.conv_layer_3(h_pool2)
    h_conv4 = self.conv_layer_4(h_conv3)
    h_conv5 = self.conv_layer_5(h_conv4)
    h_conv5_flat = self.flatten_layer(h_conv5)
    h_fc1 = self.fc_layer_1(h_conv5_flat)

    #training할 때만 dropout적용되게 하기 위해 boolean인자인 is_training사용
    h_fc1_drop = self.dropout(h_fc1, training = is_training)
    logits = self.output_layer(h_fc1_drop)
    y_pred = tf.nn.softmax(logits)

    return y_pred, logits

@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

optimizer = tf.optimizers.RMSprop(1e-3)

@tf.function
def train_step(model, x,y, is_training):
  with tf.GradientTape() as tape:
    y_pred, logits = model(x, is_training)
    loss = cross_entropy_loss(logits, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

CNN_model = CNN()

for i in range(10000):
  batch_x, batch_y = next(train_data_iter)

  if i % 100 ==0:
    train_accuracy = compute_accuracy(CNN_model(batch_x, False)[0], batch_y)
    loss_print = cross_entropy_loss(CNN_model(batch_x, False)[1], batch_y)

    print(train_accuracy, loss_print)

  train_step(CNN_model, batch_x, batch_y, True)

test_accuracy = 0.0
for i in range(10):
  test_batch_x, test_batch_y = next(test_data_iter)
  test_accuracy = test_accuracy + compute_accuracy(CNN_model(test_batch_x, False)[0], test_batch_y).numpy()
test_accuracy = test_accuracy/10
print(test_accuracy)



tf.Tensor(0.0625, shape=(), dtype=float32) tf.Tensor(2.3048513, shape=(), dtype=float32)
tf.Tensor(0.25, shape=(), dtype=float32) tf.Tensor(1.9656091, shape=(), dtype=float32)
tf.Tensor(0.390625, shape=(), dtype=float32) tf.Tensor(1.7277498, shape=(), dtype=float32)
tf.Tensor(0.3671875, shape=(), dtype=float32) tf.Tensor(1.7669963, shape=(), dtype=float32)
tf.Tensor(0.484375, shape=(), dtype=float32) tf.Tensor(1.4062089, shape=(), dtype=float32)
tf.Tensor(0.390625, shape=(), dtype=float32) tf.Tensor(1.5893736, shape=(), dtype=float32)
tf.Tensor(0.6171875, shape=(), dtype=float32) tf.Tensor(1.2298894, shape=(), dtype=float32)
tf.Tensor(0.578125, shape=(), dtype=float32) tf.Tensor(1.1573594, shape=(), dtype=float32)
tf.Tensor(0.609375, shape=(), dtype=float32) tf.Tensor(1.1268064, shape=(), dtype=float32)
tf.Tensor(0.5625, shape=(), dtype=float32) tf.Tensor(1.33607, shape=(), dtype=float32)
tf.Tensor(0.65625, shape=(), dtype=float32) tf.Tensor(0.9870999, shape=(), dtype=float32)
tf.Tenso